In [86]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from Data_cleaning import *
from helpers import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data\dataset\dataset")

In [101]:
#Turn y into 0s and 1s
y_tr_ = (y_train + 1) / 2
y_tr = y_tr_[:240000]
y_te = y_tr_[240000:]
print(y_tr.shape, y_te.shape, y_tr, y_te)

(240000,) (88135,) [0. 0. 0. ... 0. 0. 0.] [0. 0. 0. ... 0. 0. 0.]


In [102]:
def nan_corr_each_feature(X, y):
    """
    Compute absolute Pearson correlation between each column of X and y,
    ignoring NaNs. Returns |r_j| for each feature.
    """
    n_features = X.shape[1]
    corrs = np.zeros(n_features)
    
    for j in range(n_features):
        xj = X[:, j]
        
        # mask valid (non-NaN) pairs
        mask = ~np.isnan(xj) & ~np.isnan(y)
        if np.sum(mask) < 2:
            corrs[j] = np.nan
            continue
        
        x_valid = xj[mask]
        y_valid = y[mask]
        
        # compute Pearson correlation
        x_centered = x_valid - np.mean(x_valid)
        y_centered = y_valid - np.mean(y_valid)
        
        num = np.sum(x_centered * y_centered)
        den = np.sqrt(np.sum(x_centered**2) * np.sum(y_centered**2))
        corrs[j] = abs(num / den) if den != 0 else np.nan  # <-- absolute value here

    return corrs



In [103]:
# --- CLEANING ---
"""
# 0. Remove categorical (low-cardinality) features
x_num, cat_mask = remove_categorical_features(x_train, threshold=10)
print(x_num.shape)
"""
# 1. Remove features with too many NaNs
x_clean, keep_mask = remove_nan_features(x_train)
x_clean_tr = x_clean[:240000]
x_clean_te = x_clean[240000:]

# 2. Impute remaining missing values
# Compute feature means from the training data (ignore NaNs)
train_means = np.nanmean(x_clean_tr, axis=0)
train_stds = np.nanstd(x_clean_tr, axis = 0)
# Replace NaNs in training data with training means
inds_tr = np.where(np.isnan(x_clean_tr))
x_clean_tr[inds_tr] = np.take(train_means, inds_tr[1])

# Replace NaNs in test data with training means
inds_te = np.where(np.isnan(x_clean_te))
x_clean_te[inds_te] = np.take(train_means, inds_te[1])
"""
# Compute F-scores using only the training data
F_scores = anova_f_test(x_clean_tr, y_tr)


# Select top 30 features
top_k = 
top_features_idx = np.argsort(F_scores)[-top_k:][::-1]

# Apply the same feature selection to all sets
x_anova_tr = x_clean_tr[:, top_features_idx]
x_anova_te = x_clean_te[:, top_features_idx]
"""
# --- STANDARDIZATION ---
x_tr_st = (x_clean_tr - train_means) / train_stds
x_te_st = (x_clean_te - train_means) / train_stds


print(x_tr_st.shape, y_tr.shape)

(240000, 144) (240000,)


In [104]:
x_tr_ba, y_tr_ba = balance_data(x_tr_st, y_tr)
print(x_tr_ba.shape, y_tr_ba.shape)

(42262, 144) (42262,)


In [105]:
loss, w = ridge_regression(
    y_tr_ba, x_tr_ba,lambda_=1e-5)
print(loss, w, w.shape) 


0.5081081431065447 [-1.70926626e-02  2.64747576e-02 -1.35694610e-02 -1.35815254e-02
  1.05221982e-03 -1.71557880e-02  3.94386838e-03 -2.33641828e-03
 -2.33641828e-03  1.33759420e-01 -2.00081120e-02 -1.63375036e-03
 -7.18644285e-03  2.37593864e-03 -1.04251847e-02 -3.22424636e-03
 -5.85386445e-02 -4.83793257e-02 -5.42016602e-02 -6.62735293e-02
 -1.87262441e-02 -1.41425416e-02 -4.41069808e-03 -1.53855514e-03
 -1.11224887e-02  3.84590846e-03 -7.78985903e-03 -9.17470227e-03
 -2.63354832e-02 -1.01355748e-01 -9.94850164e-03  1.00494809e-03
  7.44871490e-03 -4.05707312e-03  4.23609207e-02  3.34737633e-03
  1.56486183e-02  1.31373406e-02 -1.00154625e-02  7.39979463e-03
 -1.31643008e-02 -6.43006046e-03 -7.64933507e-04 -1.50417960e-04
 -1.16622088e-02  4.67799011e-03  1.50542101e-04 -2.54615894e-02
  5.72305020e-03  1.90485568e-02 -2.19375375e-04  1.84587776e-03
  3.04779006e-03  8.84876487e-03 -2.25939932e-03 -4.00153166e-03
  1.18253185e-02  7.22852850e-03  1.51440332e-02 -4.20824981e-03
 -2.42

In [106]:
print(x_te_st.shape, w.shape)

y_pred = predict_ridge(x_te_st, w, 0.5)
print("xd")
n_errors = np.sum(np.abs(y_te_clean - y_pred))
accuracy = np.mean(y_te_clean == y_pred)

print("Misclassified samples:", n_errors)
print(f"Accuracy: {accuracy:.3f}")
print(np.sum(y_pred), np.sum(y_te_clean))

(88135, 144) (144,)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 144 is different from 145)

In [75]:
def f1_score(y_true, y_pred):
    """Compute F1 score between true and predicted labels."""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    if tp + fp == 0 or tp + fn == 0:
        return 0.0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)
    

In [76]:
print(f1_score(y_te_clean, y_pred))

0.4006347527109231
